### Toy Network Training

Below, we train a toy neural network and optimize its space using `reducible`

In [2]:
%load_ext autoreload
%autoreload 2

# toy network example 
import tensorflow as tf
import numpy as np

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

base_model = tf.keras.models.Sequential()
base_model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
base_model.add(tf.keras.layers.Dense(128, activation='relu'))
base_model.add(tf.keras.layers.Dense(128, activation='relu'))
base_model.add(tf.keras.layers.Dense(128, activation='relu'))
base_model.add(tf.keras.layers.Dropout(0.2))
base_model.add(tf.keras.layers.Dense(10))

base_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
base_model.fit(x_train, y_train, epochs=15)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch 1/15
1875/1875 [==============================] - 2s 956us/step - loss: 0.2531 - accuracy: 0.9237
Epoch 2/15
1875/1875 [==============================] - 2s 956us/step - loss: 0.1067 - accuracy: 0.9678
Epoch 3/15
1875/1875 [==============================] - 2s 959us/step - loss: 0.0781 - accuracy: 0.9761
Epoch 4/15
1875/1875 [==============================] - 2s 955us/step - loss: 0.0610 - accuracy: 0.9811
Epoch 5/15
1875/1875 [==============================] - 2s 962us/step - loss: 0.0499 - accuracy: 0.9836
Epoch 6/15
1875/1875 [==============================] - 2s 956us/step - loss: 0.0429 - accuracy: 0.9865
Epoch 7/15
1875/1875 [==============================] - 2s 951us/step - loss: 0.0365 - accuracy: 0.9883
Epoch 8/15
1875/1875 [==============================] - 2s 949us/step - loss: 0.0305 - accuracy: 0.9905
Epoch 9/15
1875/1875 [==============================] - 2s 945us/step - loss: 0.

### after training, our original model gets...

In [6]:
acc_base = base_model.evaluate(x_test,  y_test, verbose=2)[1]

313/313 - 0s - loss: 0.1167 - accuracy: 0.9758 - 127ms/epoch - 406us/step


In [8]:
from reducible.optimize import optimize_model

optimized_model = optimize_model(base_model, x_test, y_test)

313/313 - 0s - loss: 1.3578 - accuracy: 0.5638 - 377ms/epoch - 1ms/step
313/313 - 0s - loss: 1.3616 - accuracy: 0.5634 - 349ms/epoch - 1ms/step
313/313 - 0s - loss: 1.3630 - accuracy: 0.5635 - 364ms/epoch - 1ms/step
313/313 - 0s - loss: 1.3573 - accuracy: 0.5641 - 355ms/epoch - 1ms/step
313/313 - 0s - loss: 0.4534 - accuracy: 0.8513 - 347ms/epoch - 1ms/step
313/313 - 0s - loss: 0.4541 - accuracy: 0.8511 - 353ms/epoch - 1ms/step
313/313 - 0s - loss: 0.4535 - accuracy: 0.8522 - 357ms/epoch - 1ms/step
313/313 - 0s - loss: 0.4573 - accuracy: 0.8519 - 351ms/epoch - 1ms/step
313/313 - 0s - loss: 0.3196 - accuracy: 0.9025 - 363ms/epoch - 1ms/step
313/313 - 0s - loss: 0.3245 - accuracy: 0.9006 - 362ms/epoch - 1ms/step
313/313 - 0s - loss: 0.3332 - accuracy: 0.8972 - 354ms/epoch - 1ms/step
313/313 - 0s - loss: 0.3286 - accuracy: 0.9006 - 370ms/epoch - 1ms/step
313/313 - 0s - loss: 0.2070 - accuracy: 0.9506 - 358ms/epoch - 1ms/step
313/313 - 0s - loss: 0.2103 - accuracy: 0.9509 - 355ms/epoch - 1

### and our parameter-reduced model gets ... 

In [9]:
acc_opt = optimized_model.evaluate(x_test,  y_test, verbose=2)[1]

313/313 - 0s - loss: 0.1451 - accuracy: 0.9667 - 227ms/epoch - 725us/step


### Calculating the percent difference in accuracy

In [10]:
100*(acc_base - acc_opt)/acc_base

0.9325641047724825

### Saving our model to disk

In [11]:
from reducible.save import save_model_to_file, load_model_from_file

save_model_to_file(optimized_model, "optimized_model.gyatt")
tf.keras.models.save_model(base_model, "base_model.keras")

In [12]:
loaded_model = load_model_from_file("optimized_model.gyatt")

In [13]:
# checking to make sure model still works 

loaded_model.evaluate(x_test,  y_test, verbose=2);

313/313 - 0s - loss: 0.1451 - accuracy: 0.9667 - 187ms/epoch - 599us/step


## Future improvements: include serialization optimizations to close the gap

In [15]:
opt_total = 0
base_total = 0
# calculates predicted reduction by simply calculating the number of terms in weights and bias matrices 

from reducible.optimize import RankKApprox

for layer in optimized_model.layers: 
    if isinstance(layer, RankKApprox):
        opt_total += layer.A.numpy().size + layer.B.numpy().size +  layer.b.numpy().size

for layer in base_model.layers: 
    if layer.get_weights(): 
        w, b = layer.get_weights()
        base_total += w.size
        base_total += b.size

predicted_reduction = 1-opt_total/base_total

In [16]:
import os
actual_reduction = 1-os.path.getsize('optimized_model.gyatt')/os.path.getsize('base_model.keras')

In [17]:
print(f"Predicted reduction {predicted_reduction*100:0.1f}% | Actual reduction {actual_reduction*100:0.1f}%")

Predicted reduction 43.4% | Actual reduction 41.9%
